In [135]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from pandas import DataFrame
from pandas import Series
from netCDF4 import Dataset
from netCDF4 import num2date
import datetime
import os
import sys
import os.path
import glob

In [136]:
# Reading in model output files, from three different applications, fabm0d, gotm1d and gotmlake
output_dir= os.path.normpath(os.getcwd() + os.sep + os.pardir+"/output")
# fabm0d files
fabm0d=[]
for f in glob.glob("*fabm0d*.nc"):
    fabm0d.append(f)
# gotm1d files
gotm1d=[]
for f in glob.glob("*gotm1d*.nc"):
    gotm1d.append(f)
# gotm-lake files
gotmlake=[]
for f in glob.glob("*gotmlake*.nc"):
    gotmlake.append(f)

In [137]:
# define the start of the time and the end of the time for plot range
start=valid_times.index(datetime.datetime(2015, 1, 1, 0, 0))
stop=valid_times.index(datetime.datetime(2016, 1, 1, 0, 0))


In [138]:
# Read in gotm1d data, get the vertical avarage value, and store in nc Dataset form
# creat dict for store output of wanted fabm0d data
df_fabm0d={}
for f in fabm0d:
    path=os.path.join(output_dir, f)
    fabm0d_nc=Dataset(path, mode='r')
    time        = fabm0d_nc.variables['time']
    units       = fabm0d_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
    time=valid_times[start:stop]
    # create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# loop over time and put the variable in lists
    i=0
    for t in time:
        temperature.append(fabm0d_nc.variables['temp'][i,0,0])
        Oxygen.append(fabm0d_nc.variables['abiotic_water_sO2W'][i,0,0])
        TP.append(fabm0d_nc.variables['pclake_totP_calculator_result'][i,0,0])
        TN.append(fabm0d_nc.variables['pclake_totN_calculator_result'][i,0,0])
        Phyto.append(fabm0d_nc.variables['phytoplankton_water_aDPhytW'][i,0,0])
        Zoo.append(fabm0d_nc.variables['foodweb_water_sDZoo'][i,0,0])
        Fish.append(fabm0d_nc.variables['foodweb_water_sDFiAd'][i,0,0]+
                    fabm0d_nc.variables['foodweb_water_sDFiJv'][i,0,0])
        PAR.append(fabm0d_nc.variables['phytoplankton_water_partop'][i,0,0])
        Veg.append(fabm0d_nc.variables['macrophytes_sDVeg'][i,0,0])
        Ben.append(fabm0d_nc.variables['foodweb_sediment_sDBent'][i,0,0])
        i=i+1
# put list into pandas dataframe format
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])   
# put data into dict for future usage
    df_fabm0d[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)
    

In [144]:
# Read in gotm1d data, get the vertical avarage value, and store in nc Dataset form
# creat dict for store output of wanted GOTM1D data
df_gotm1d={}
# Loop over gotm1d files and get the netcdf dataa
for f in gotm1d:
    path=os.path.join(output_dir, f)
    gotm1d_nc=Dataset(path, mode='r')
    time        = gotm1d_nc.variables['time']
    units       = gotm1d_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
    time=valid_times[start:stop]
# create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# loop over time and put the variable in lists
    i=0
    for t in time:
        temperature.append(np.mean(gotm1d_nc.variables['temp'][i,:,0,0]))
        Oxygen.append(np.mean(gotm1d_nc.variables['abiotic_water_sO2W'][i,:,0,0]))
        TP.append(np.mean(gotm1d_nc.variables['pclake_totP_calculator_result'][i,:,0,0]))
        TN.append(np.mean(gotm1d_nc.variables['pclake_totN_calculator_result'][i,:,0,0]))
        Phyto.append(np.mean(gotm1d_nc.variables['phytoplankton_water_aDPhytW'][i,:,0,0]))
        Zoo.append(np.mean(gotm1d_nc.variables['foodweb_water_sDZoo'][i,:,0,0]))
        Fish.append(np.mean(gotm1d_nc.variables['foodweb_water_sDFiAd'][i,:,0,0])+
                    np.mean(gotm1d_nc.variables['foodweb_water_sDFiJv'][i,:,0,0]))
        PAR.append(gotm1d_nc.variables['phytoplankton_water_partop'][i,-1,0,0])
        Veg.append(gotm1d_nc.variables['macrophytes_sDVeg'][i,0,0])
        Ben.append(gotm1d_nc.variables['foodweb_sediment_sDBent'][i,0,0])
        i=i+1
# put list into pandas dataframe format
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])
# put data into dict for future usage
    df_gotm1d[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)


In [145]:
# Read in gotm1d data, store in nc Dataset form
# Read in gotm1d_2m
# Create empty dict for storing output of wanted gotmlake data
df_gotmlake={}
# Get the volumn fraction for each depth in gotmlake
# calculated according to hypsograph data in excel.
f_Vn_2m=[0.04,0.12,0.20,0.28,0.36]
f_Vn_5m=[0.01,0.02,0.03,0.04,0.05,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.15]
f_Vn_10m=[0.0025,0.0056,0.0087,0.0119,0.0150,0.0181,0.0212,0.0244,0.0275,
          0.0306,0.0337,0.0369,0.0400,0.0431,0.0463,0.0494,0.0525,0.0556,
          0.0588,0.0619,0.0650,0.0681,0.0713,0.0744,0.0775]
f_Vn_20m=[0.000854443,0.001635895,0.002417346,0.003198797,0.003980249,0.0047617,
          0.005543151,0.006324602,0.007106054,0.007887505,0.008668956,0.009450408,
          0.010231859,0.01101331,0.011794761,0.012576213,0.013357664,0.014139115,
          0.014920567,0.015702018,0.016483469,0.01726492,0.018046372,0.018827823,
          0.019609274,0.020390726,0.021172177,0.021953628,0.02273508,0.023516531,
          0.024297982,0.025079433,0.025860885,0.026642336,0.027423787,0.028205239,
          0.02898669,0.029768141,0.030549592,0.031331044,0.032112495,0.032893946,
          0.033675398,0.034456849,0.0352383,0.036019751,0.036801203,0.037582654,
          0.038364105,0.039145557]
# Loop over gotmlake files and get the netcdf data
for f in gotmlake:
    path=os.path.join(output_dir, f)
    gotmlake_nc=Dataset(path, mode='r')
    time        = gotmlake_nc.variables['time']
    units       = gotmlake_nc.variables['time'].units
    valid_times = num2date(time[:], units=units).tolist()
    time=valid_times[start:stop]
# Get the f_lvl for different depth
    lvl=len(gotmlake_nc.variables['temp'][0])
    if lvl==5:
        f_lvl=f_Vn_2m
    elif lvl== 13:
        f_lvl=f_Vn_5m
    elif lvl==25:
        f_lvl=f_Vn_10m
    elif lvl==50:
        f_lvl=f_Vn_20m
    else:
        print "I can't find f_lvl"      
# create empty lists for storging extracted and treated variables
    temperature=[]
    PAR=[]
    Oxygen=[]
    TP=[]
    TN=[]
    Phyto=[]
    Zoo=[]
    Fish=[] 
    Veg=[]
    Ben=[]
# Loop over time, treat the data and put variables in lists
    i=0
    for t in time:      
# get the volume average data from gotmlake
        temperature.append(np.sum(gotmlake_nc.variables['temp'][i,:,0,0]*f_lvl[:]))
        Oxygen.append(np.sum(gotmlake_nc.variables['abiotic_water_sO2W'][i,:,0,0]*f_lvl[:]))
        TP.append(np.sum(gotmlake_nc.variables['pclake_totP_calculator_result'][i,:,0,0]*f_lvl[:]))
        TN.append(np.sum(gotmlake_nc.variables['pclake_totN_calculator_result'][i,:,0,0]*f_lvl[:]))
        Phyto.append(np.sum(gotmlake_nc.variables['phytoplankton_water_aDPhytW'][i,:,0,0]*f_lvl[:]))
        Zoo.append(np.sum(gotmlake_nc.variables['foodweb_water_sDZoo'][i,:,0,0]*f_lvl[:]))
        Fish.append(np.sum(gotmlake_nc.variables['foodweb_water_sDFiAd'][i,:,0,0]*f_lvl[:])+
                           np.sum(gotmlake_nc.variables['foodweb_water_sDFiJv'][i,:,0,0]*f_lvl[:]))
# PAR is stil the top layer data
        PAR.append(gotmlake_nc.variables['phytoplankton_water_partop'][i,-1,0,0])
# Get gotmlake sediment data, now it's 1D for state variables
        Veg.append(np.mean(gotmlake_nc.variables['macrophytes_sDVeg'][i,:,0,0]))
        Ben.append(np.mean(gotmlake_nc.variables['foodweb_sediment_sDBent'][i,:,0,0]))
        i=i+1
# Convert data into pandas dataframework
    temp=DataFrame(temperature,index=time,columns=['temp'],)
    PAR=DataFrame(PAR,index=time,columns=['PAR'])
    O2=DataFrame(Oxygen,index=time,columns=['O2'])
    totP=DataFrame(TP,index=time,columns=['totP'])
    totN=DataFrame(TN,index=time,columns=['totN'])
    aDPhytW=DataFrame(Phyto,index=time,columns=['aDPhytW'])
    sDZoo=DataFrame(Zoo,index=time,columns=['sDZoo'])
    aDFish=DataFrame(Fish,index=time,columns=['aDFish'])
    sDVeg=DataFrame(Veg,index=time,columns=['sDVeg'])
    sDBent=DataFrame(Ben,index=time,columns=['sDBent'])
# Store gotmlake data into dict for future usage
    df_gotmlake[f]=pd.concat([temp,PAR,O2,totP,totN,aDPhytW,sDZoo,aDFish,sDVeg,sDBent],axis=1)

In [146]:
# join dataframe according to depths
df_2m= pd.concat([df_fabm0d['pclake-fabm0d-2m.nc'],df_gotm1d['pclake-gotm1d-2m.nc'],
                  df_gotmlake['pclake-gotmlake-2m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_5m= pd.concat([df_fabm0d['pclake-fabm0d-5m.nc'],df_gotm1d['pclake-gotm1d-5m.nc'],
                  df_gotmlake['pclake-gotmlake-5m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_10m= pd.concat([df_fabm0d['pclake-fabm0d-10m.nc'],df_gotm1d['pclake-gotm1d-10m.nc'],
                   df_gotmlake['pclake-gotmlake-10m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
df_20m= pd.concat([df_fabm0d['pclake-fabm0d-20m.nc'],df_gotm1d['pclake-gotm1d-20m.nc'],
                   df_gotmlake['pclake-gotmlake-20m.nc']],axis=1,keys=['fabm0d', 'gotm1d', 'gotmlake'])
# Joined all data together for plot
results= pd.concat([df_2m,df_5m,df_10m,df_20m],axis=1,keys=['2m','5m','10m','20m'])

In [147]:
models =list(set(list(df_2m.columns.get_level_values(0))))
#variables=list(set(list(df_2m.columns.get_level_values(1))))
variables = ['temp', 'totN', 'totP',  'O2']
for var in variables:
    for model in models:
#        plt.plot(df_2m.index,df_2m[model][var],label= model)
        i=i+1

#print models
#print variables
# plt.plot(df_2m.index,df_2m[model][var],label= model)
#plt.show()
#df_2m[model][var]

In [73]:
variables = ['temp',  'totP',  'O2']
# experiment with plot
fig = plt.figure(figsize=(8.27, 11.69), dpi=100)
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(8.27, 11.69))
i=0
for var in variables:
    axs[i].plot(time,df_2m['fabm0d'][var])
    i=i+1
#    print df_2m['fabm0d'][variables[i]]
fig.show()

In [148]:
# Make the export pdf file
fig_pdf='fig1.pdf'
pdf_Output = os.path.join(output_dir,fig_pdf)
# Get the index in the DataFrame for iteration
# Get multi-index for 2m
#
with PdfPages(pdf_Output) as pdf:

#    models =list(df_2m.columns.get_level_values(0))
#    variables=list(df_2m.columns.get_level_values(1))
    fig = plt.figure(figsize=(8.27, 11.69), dpi=100)
 
            
    models=['fabm0d','gotm1d','gotmlake']
    variables = ['temp', 'totN', 'totP',  'O2']
    nplots = len(variables)
    fig, axs = plt.subplots(nrows=1, ncols=nplots, figsize=(8.27, 11.69))  #sharex=True,
    plt.subplots_adjust(hspace = .1)

#       variables = ['totp', 'pho4', 'totn', 'nitr', 'ammo', 'chla', 'temp', 'o2']

    for i in range(len(variables)):
        
        #pos = ax.get_position()
        #x = pos.x0 + 0.7
        #y = pos.y0 + 0.06
        for var in variables:
            axs[i].plot(time,df_5m['fabm0d'][var],label= 'fabm0d', color='black')
            axs[i].plot(time,df_5m['gotm1d'][var], label='gotm1d', color='grey')
            axs[i].plot(time,df_5m['gotmlake'][var], label='gotmlake', color='grey')
            #        axs[i].plot(df_2m.index, df_2m['fabm0d']['temp'], label='fabm0d', color='black')
#        axs[i].axvspan(*mdates.datestr2num(['01/01/2003', '12/31/2013']), color='red', alpha=0.5)
#        axs[i].axvspan(*mdates.datestr2num(['01/01/1989', '12/31/1993']), color='grey', alpha=0.5)
#        axs[i].plot(result['time_x'], result['%s' %(variables[i])+'_y'], label='Obs',    marker='o',
#                                                                                        color='grey',
#                                                                                        markersize=2 )
#        Label = axs[i].set_ylabel("var =" '%s' %(variables[i]), **axis_font)
#        plt.xticks(rotation=90)
#        axs[i].xaxis.set_major_locator(mdates.YearLocator())
#        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        #axs[i].xaxis.set_minor_locator(mdates.MonthLocator())

        #plt.legend(frameon=False, loc='upper center', bbox_to_anchor=(0.5, -0.05), numpoints=1)

        #plt.tight_layout(pad=2, w_pad=2, h_pad=1)
        
#        plt.show()


    pdf.savefig()  # saves the current figure into a pdf page
    plt.close()

    

In [33]:
variables=['temp','O2','totN','totP']
models=['fabm0d']
j=0
for var in variables:
#    print variables[j]
#    print df_2m[models][variables]
#    print variable
    j=j+1
print variables[0]
df_2m[models]['temp']


temp


KeyError: 'temp'

In [23]:
# Plot three model outputs, line plots
# plot Tm, O2, TN and TP
variables=['temp','O2','totN','totP']
fig=plt.subplots(4,4,sharex=True)
plt.subplots_adjust(hspace=0)
for i in range(16):
    fig[i].plot(time, df_2m['fabm0d'].variables)


AttributeError: 'Figure' object has no attribute 'plot'

#possible code for surface output
if f[-6:-5]=='2m':
lvl=4
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='5m':
lvl=12
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='10m':
lvl=24
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])
elif f[-6:-5]=='20m':
lvl=49
print lvl
PAR.append(gotm1d_nc.variables['rad'][i,lvl,0,0])